## Import packages

In [ ]:
import requests
import pandas as pd
import os
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime

## Set the quarter and adjust waterfall settings if needed

In [ ]:
last_month = 6
last_year = 2024

nbr_months_waterfall = 3

# Output file name
outputfile = "Waterfall_ElectricityGeneration_20232024Q2_Python"

# Define which categories should have labels outside, edit in case other fuels need label adjustments
outside_labels = ['Nuclear', 'Other']

# Define y-axis limits and ticks
y_min, y_max = -50, 30  # Adjusted to go a bit above and below the ticks
y_ticks = np.arange(-50, 26, 25)

## API access parameters

In [ ]:
my_api_key = "51356ef8-f7dd-43a1-9629-1965b7830a26"
base_url = "https://api.ember-climate.org"

## Parameters for the charts

### Countries to import

In [ ]:
iso_codes = [
    "ALB",  # Albania
    "AUT",  # Austria
    "BEL",  # Belgium
    "BIH",  # Bosnia and Herzegovina
    "BGR",  # Bulgaria
    "HRV",  # Croatia
    "CYP",  # Cyprus
    "CZE",  # Czech Republic
    "DNK",  # Denmark
    "EST",  # Estonia
    "FIN",  # Finland
    "FRA",  # France
    "DEU",  # Germany
    "GRC",  # Greece
    "HUN",  # Hungary
    "IRL",  # Ireland
    "ITA",  # Italy
    "XKX",  # Kosovo
    "LVA",  # Latvia
    "LTU",  # Lithuania
    "LUX",  # Luxembourg
    "MLT",  # Malta
    "MDA",  # Moldova
    "MNE",  # Montenegro
    "NLD",  # Netherlands
    "MKD",  # North Macedonia
    "NOR",  # Norway
    "POL",  # Poland
    "PRT",  # Portugal
    "ROU",  # Romania
    "SRB",  # Serbia
    "SVK",  # Slovakia
    "SVN",  # Slovenia
    "ESP",  # Spain
    "SWE",  # Sweden
    "CHE",  # Switzerland
    "TUR",  # Turkey
    "GBR",  # United Kingdom
]

#### Listing the months we will have, based on the parameters we have chosen

In [ ]:
months_old_waterfall = [
    f"{year}-{month:02d}-01"
    for year, month in [
        time.localtime(
            time.mktime((last_year - 1, last_month - n, 1, 0, 0, 0, 0, 0, 0))
        )[:2]
        for n in range(nbr_months_waterfall)
    ]
]
months_new_waterfall = [
    f"{year}-{month:02d}-01"
    for year, month in [
        time.localtime(
            time.mktime((last_year, last_month  - n, 1, 0, 0, 0, 0, 0, 0))
        )[:2]
        for n in range(nbr_months_waterfall)
    ]
]
months_expected = [
    f"{month:02d}"
    for year, month in [
        time.localtime(
            time.mktime((last_year, last_month  - n, 1, 0, 0, 0, 0, 0, 0))
        )[:2]
        for n in range(nbr_months_waterfall)
    ]
]

In [ ]:
months_old_waterfall

In [ ]:
months_new_waterfall

## Running the API

### Queries

In [ ]:
query_url_generation = (
    f"{base_url}/v1/electricity-generation/monthly"
    + f"?entity_code={','.join(iso_codes)}&is_aggregate_series=false&start_date=2000&api_key={my_api_key}"
)

query_url_demand = (
    f"{base_url}/v1/electricity-demand/monthly"
    + f"?entity_code={','.join(iso_codes)}&is_aggregate_series=false&start_date=2000&api_key={my_api_key}"
)

In [ ]:
query_url_generation

In [ ]:
query_url_demand

### Executing the queries. 200 = success

In [ ]:
response_generation = requests.get(query_url_generation)
response_demand = requests.get(query_url_demand)

In [ ]:
response_generation.status_code

In [ ]:
response_demand.status_code

## Extracting the results

In [ ]:
data_generation = response_generation.json()["data"]
data_demand = response_demand.json()["data"]
df_generation = pd.DataFrame(data_generation)
df_demand = pd.DataFrame(data_demand)

In [ ]:
df_generation

In [ ]:
df_demand

### Filtering only the data that we need

#### We extract the month from the date

In [ ]:
df_generation["month"] = df_generation["date"].str[5:7]
df_demand["month"] = df_demand["date"].str[5:7]

#### We create one table with the current ('new') data, one table with the data we compare with ('old')

In [ ]:
df_generation_old = df_generation.loc[
        df_generation["date"].isin(months_old_waterfall)
    ].copy()
df_generation_old["generation_twh_old"] = df_generation_old["generation_twh"]
df_generation_old = df_generation_old[
        ["entity", "entity_code", "month", "series", "generation_twh_old"]
    ]
df_generation_new = df_generation.loc[
        df_generation["date"].isin(months_new_waterfall)
    ].copy()
df_generation_new["generation_twh_new"] = df_generation_new["generation_twh"]
df_generation_new = df_generation_new[
        ["entity", "entity_code", "month", "series", "generation_twh_new", "date"]
    ]

In [ ]:
df_demand_old = df_demand.loc[df_demand["date"].isin(months_old_waterfall)].copy()
df_demand_old["series"] = "Demand"
df_demand_old["generation_twh_old"] = df_demand_old["demand_twh"]
df_demand_old = df_demand_old[
        ["entity", "entity_code", "month", "series", "generation_twh_old"]
    ]

df_demand_new = df_demand.loc[df_demand["date"].isin(months_new_waterfall)].copy()
df_demand_new["series"] = "Demand"
df_demand_new["generation_twh_new"] = df_demand_new["demand_twh"]
df_demand_new = df_demand_new[
        ["entity", "entity_code", "month", "series", "generation_twh_new", "date"]
    ]

#### We merge generation and demand tables

In [ ]:
df_old = pd.concat([df_generation_old, df_demand_old])
df_new = pd.concat([df_generation_new, df_demand_new])

#### We merge the 'old' and 'new'

In [ ]:
df_all = pd.merge(
        df_old, df_new, how="outer", on=["entity", "entity_code", "series", "month"]
    )

#### We rename some categories

In [ ]:
df_all["series"] = df_all["series"].apply(
        lambda x: (
            "Other"
            if x in ["Bioenergy", "Net imports", "Other renewables", "Other fossil"]
            else x
        )
    )
df_all = df_all.fillna(0)

### Data checks

In [ ]:
combinations = [(x, y) for x in iso_codes for y in months_expected]
expected_outcome = pd.DataFrame(combinations, columns=['entity_code', 'month'])

#### Missing countries in generation - Old

In [ ]:
df_generation_old_check = df_generation_old[df_generation_old.generation_twh_old > 0]
df_generation_old_check = df_generation_old_check[['entity_code','month']].drop_duplicates()

expected_outcome[~expected_outcome.set_index(['entity_code','month']).index.isin(df_generation_old_check.set_index(['entity_code','month']).index)]

#### Missing countries in demand - Old

In [ ]:
df_demand_old_check = df_demand_old[df_demand_old.generation_twh_old > 0]
df_demand_old_check = df_demand_old_check[['entity_code','month']].drop_duplicates()

expected_outcome[~expected_outcome.set_index(['entity_code','month']).index.isin(df_demand_old_check.set_index(['entity_code','month']).index)]

#### Missing countries in generation - New

In [ ]:
df_demand_new_check = df_demand_new[df_demand_new.generation_twh_new > 0]
df_demand_new_check = df_demand_new_check[['entity_code','month']].drop_duplicates()

expected_outcome[~expected_outcome.set_index(['entity_code','month']).index.isin(df_demand_new_check.set_index(['entity_code','month']).index)]

#### Missing countries in demand - New

In [ ]:
df_generation_new_check = df_generation_new[df_generation_new.generation_twh_new > 0]
df_generation_new_check = df_generation_new_check[['entity_code','month']].drop_duplicates()

expected_outcome[~expected_outcome.set_index(['entity_code','month']).index.isin(df_generation_new_check.set_index(['entity_code','month']).index)]

### Final data extraction

#### We extract what we need

In [ ]:
output_waterfall = (
        df_all.loc[df_all["date"].isin(months_new_waterfall)][
            ["series", "generation_twh_old", "generation_twh_new"]
        ]
        .groupby(["series"])
        .sum()
        .reset_index()
    )
output_waterfall["y-o-y change"] = (
        output_waterfall["generation_twh_new"] - output_waterfall["generation_twh_old"]
    )

In [ ]:
cwd = os.getcwd()
excel_path_waterfall = os.path.join(cwd,"data_waterfall.xlsx")
output_waterfall.to_excel(excel_path_waterfall, index=False)

In [ ]:
output_waterfall

# Waterfall graphic

## Define source data, labels and colors

In [ ]:
plt.close('all')
matplotlib.use('Agg')
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['text.usetex'] = False
#plt.rcParams['font.family'] = 'Open Sans'  # Or 'Montserrat'

In [ ]:
# Load the data
data = output_waterfall

# Define quarter title, subtitle and axis title label
old_from = datetime.strptime(months_old_waterfall[-1], '%Y-%m-%d').strftime('%B %Y')
old_to = datetime.strptime(months_old_waterfall[0], '%Y-%m-%d').strftime('%B %Y')
new_from = datetime.strptime(months_new_waterfall[-1], '%Y-%m-%d').strftime('%B %Y')
new_to = datetime.strptime(months_new_waterfall[0], '%Y-%m-%d').strftime('%B %Y')
title = "EUROPE: year-on-year change in electricity generation by fuel in the last quarter"
months_old_waterfall
subtitle = f"{old_from} - {old_to} versus {new_from} - {new_to}"
month_from = datetime.strptime(months_new_waterfall[-1], '%Y-%m-%d').strftime('%B')
month_to = datetime.strptime(months_new_waterfall[0], '%Y-%m-%d').strftime('%B')
quarter = f"{month_from}-{month_to}"

## Chart

In [ ]:
# Define the order you want
order = ["Coal", "Gas", "Nuclear", "Hydro", "Solar", "Wind", "Other", "Demand"]
colors = ["#475865", "#8B969F", "#FFA401", "#76C8F1", "#FEC814", "#1FDA4B", "#98C4D1", "#E0187B"]

# Ensure 'series' is treated as a categorical type with the defined order
data['series'] = pd.Categorical(data['series'], categories=order, ordered=True)

# Sort the DataFrame based on the categorical order and reset the index
data = data.sort_values('series').reset_index(drop=True)

# Define the correct colors

data['Color'] = colors[:len(data)]

# Calculate the cumulative sum for positioning each bar
data['Cumulative'] = data['y-o-y change'].cumsum()
data['Position'] = data['Cumulative'] - data['y-o-y change']

# Manually adjust the 'Position' and 'Cumulative' for "Demand"
for i, row in data.iterrows():
    if row['series'] == "Demand":
        if row['y-o-y change'] > 0:
            data.at[i, 'Position'] = data.at[i, 'Cumulative'] - row['y-o-y change']
            data.at[i, 'y-o-y change'] = -row['y-o-y change']  # Invert the value for plotting
        else:
            data.at[i, 'Position'] = data.at[i, 'Cumulative'] - row['y-o-y change']


# Plot the waterfall chart
fig, ax = plt.subplots(figsize=(7.2, 4))

bar_width = 0.8

# Plot bars and connecting lines
for i, row in data.iterrows():
    bar = ax.bar(row['series'], row['y-o-y change'], bottom=row['Position'], color=row['Color'], width=bar_width)
    if i < len(data) - 1:  # Draw connecting line to the next bar
        next_row = data.iloc[i + 1]
        line_position = row['Position'] + row['y-o-y change']
        ax.plot([i - bar_width / 2, i + bar_width / 2 + 1], [line_position, line_position], color='black')

# Add labels and values
for i, row in data.iterrows():
    if row['series'] == "Demand":
        value_text = f"+{abs(row['y-o-y change']):.1f}"
    else:
        value_text = f"+{row['y-o-y change']:.1f}" if row['y-o-y change'] > 0 else f"{row['y-o-y change']:.1f}"
    
    if row['series'] in outside_labels:
        if row['y-o-y change'] > 0:
            value_position = row['Position'] + row['y-o-y change'] + 1
            label_position = value_position + 3
            ax.text(i, value_position, value_text, ha='center', va='bottom', fontsize=8, color='black')
            ax.text(i, label_position, row['series'], ha='center', va='bottom', fontsize=8, color='black')
        else:
            line_position = row['Position'] + row['y-o-y change']
            label_position = line_position - 4
            value_position = label_position - 3
            ax.text(i, label_position, row['series'], ha='center', va='bottom', fontsize=8, color='black')
            ax.text(i, value_position, value_text, ha='center', va='bottom', fontsize=8, color='black')
    else:
        ax.text(i, row['Position'] + row['y-o-y change'] / 2 + 2, row['series'], ha='center', va='center', fontsize=8, color='black')
        ax.text(i, row['Position'] + row['y-o-y change'] / 2 - 2, value_text, ha='center', va='center', fontsize=8, color='black')



# Customize plot
ax.axhline(0, color='gray', linewidth=0.8, zorder=0)
ax.set_ylim(y_min, y_max)
ax.set_yticks(y_ticks)
ax.set_yticklabels([f'{tick}' for tick in y_ticks], fontsize=8)
ax.set_xlabel("")
ax.set_ylabel(f"Change in electricity generation {quarter} TWh", fontsize=8, labelpad=20, ha='center', weight='bold')
ax.yaxis.set_label_position("left")
ax.yaxis.set_ticks_position('left')  # Add y-axis ticks
ax.yaxis.set_tick_params(width=2, color='gray', length=10, direction='inout', labelsize=8)  # Customize y ticks appearance

plt.xticks(rotation=45, ha='right', fontsize=8)
plt.grid(False)  # Remove grid
# Remove all axis lines
for spine in ax.spines.values():
    spine.set_visible(False)
ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)  # Remove x axis ticks
ax.tick_params(axis='y', which='both', direction='in', length = 10, width = 0.5)  # Remove x axis ticks

# Add title and subtitle 
fig.suptitle(title, fontsize=12, weight='bold', ha='left', fontname='Montserrat', x=0.01, y=0.98)
fig.text(0.01, 0.9, subtitle, fontsize=10, ha='left', fontname='Open sans')

pdffile = outputfile+'.pdf'
plt.tight_layout(rect=[0, 0, 1, 1])
extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
fig.savefig(pdffile, format='pdf', bbox_inches='tight')

%matplotlib inline
plt.show()